# Загрузка данных

# git git git


second commit 

In [48]:
!pip install catboost

In [79]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from catboost import cv, Pool
from sklearn.metrics import *

In [ ]:
#weijsbg ass

In [ ]:
#wesfdfsef

In [57]:
df = pd.read_csv('data/aug_train.csv')
test_for_k = pd.read_csv('data/aug_test.csv')

In [58]:
df.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [59]:
test_for_k.head(4)

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,32403,city_41,0.827,Male,Has relevent experience,Full time course,Graduate,STEM,9,<10,NaN,1,21
1,9858,city_103,0.920,Female,Has relevent experience,no_enrollment,Graduate,STEM,5,NaN,Pvt Ltd,1,98
2,31806,city_21,0.624,Male,No relevent experience,no_enrollment,High School,NaN,<1,NaN,Pvt Ltd,never,15
3,27385,city_13,0.827,Male,Has relevent experience,no_enrollment,Masters,STEM,11,10/49,Pvt Ltd,1,39


# Предобработка

In [60]:
df = df.drop('enrollee_id', axis = 1)
test_for_k = test_for_k.drop('enrollee_id', axis = 1)

In [61]:
old_shape = df.shape[0]

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city                    19158 non-null  object 
 1   city_development_index  19158 non-null  float64
 2   gender                  14650 non-null  object 
 3   relevent_experience     19158 non-null  object 
 4   enrolled_university     18772 non-null  object 
 5   education_level         18698 non-null  object 
 6   major_discipline        16345 non-null  object 
 7   experience              19093 non-null  object 
 8   company_size            13220 non-null  object 
 9   company_type            13018 non-null  object 
 10  last_new_job            18735 non-null  object 
 11  training_hours          19158 non-null  int64  
 12  target                  19158 non-null  float64
dtypes: float64(2), int64(1), object(10)
memory usage: 1.9+ MB


In [63]:
df.isna().mean().sort_values(ascending = False)

city                      0.000000
city_development_index    0.000000
relevent_experience       0.000000
training_hours            0.000000
target                    0.000000
experience                0.003393
enrolled_university       0.020148
last_new_job              0.022080
education_level           0.024011
major_discipline          0.146832
gender                    0.235306
company_size              0.309949
company_type              0.320493
dtype: float64

In [65]:
cat_features = list(df.select_dtypes(include='object').columns)
df[cat_features] = df[cat_features].fillna('no info')
test_for_k[cat_features] = test_for_k[cat_features].fillna('no info')

In [66]:
df.shape[0] / old_shape

1.0

# Выборки

In [67]:
train, test = train_test_split(df,train_size=0.7,random_state=2003)

In [68]:
floats = list(df.select_dtypes(include='float').columns)


In [69]:
cat_features = list(df.select_dtypes(include='object').columns)

In [70]:
df.columns

Index(['city', 'city_development_index', 'gender', 'relevent_experience',
       'enrolled_university', 'education_level', 'major_discipline',
       'experience', 'company_size', 'company_type', 'last_new_job',
       'training_hours', 'target'],
      dtype='object')

In [71]:
X = ['city', 'city_development_index', 'gender', 'relevent_experience',
       'enrolled_university', 'education_level', 'major_discipline',
       'experience', 'company_size', 'company_type', 'last_new_job',
       'training_hours']
y = ['target']

# Cоздание pool

In [72]:
train_data = Pool(data=train[X],
                  label=train[y],
                  cat_features=cat_features
                 )

In [73]:
test_data = Pool(data=test[X],
                  label=test[y],
                  cat_features=cat_features
                 )

# Запуск кросс-валидации

In [74]:
params = {'cat_features': cat_features,
              'eval_metric': 'AUC',
          'loss_function': 'Logloss',
              'learning_rate': 0.005,
              'random_seed': 2003,
              'verbose':100}

In [75]:
cv_data = cv(
    params = params,
    pool = train_data,
    fold_count=5,
    shuffle=True,
    partition_random_seed=0,
    stratified=True,
    verbose=False,
    early_stopping_rounds=200,
    #plot = True
)

Training on fold [0/5]

bestTest = 0.8082348718
bestIteration = 999

Training on fold [1/5]

bestTest = 0.8054664832
bestIteration = 999

Training on fold [2/5]

bestTest = 0.79259557
bestIteration = 26

Training on fold [3/5]

bestTest = 0.8067280369
bestIteration = 999

Training on fold [4/5]

bestTest = 0.8046457629
bestIteration = 994



В таблице cv_data показаны средние значения метрик по количеству фолдов, которые мы выбрали. В данном случае 5.

In [76]:
cv_data[cv_data['test-AUC-mean'] == cv_data['test-AUC-mean'].max()]

,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
999,999,0.80326,0.006862,0.437357,0.01457,0.426136,0.018077


Если мы подобрали learning_rate таким образом (чтобы его подобрать позапускай кросс-валидацию несколько раз меняя learning_rate, но скорее всего он и сам нормально подберется), что лучшая интерация случается ближе к 1000, то ее можно и не запоминать, а просто обучать модель на дефултной 1000 итераций с тем learning_rate который мы нашли

# Разделим тренировочную выборку на тренировочную и валидационную

In [77]:
train_, val = train_test_split(train, random_state = 2003)

In [80]:
model = CatBoostClassifier(**params)

In [81]:
model.fit(train_[X],train_[y])

0:	total: 58.9ms	remaining: 58.8s
100:	total: 7.42s	remaining: 1m 6s
200:	total: 15.3s	remaining: 1m
300:	total: 23s	remaining: 53.3s
400:	total: 30.4s	remaining: 45.3s
500:	total: 37.9s	remaining: 37.8s
600:	total: 45.8s	remaining: 30.4s
700:	total: 53.5s	remaining: 22.8s
800:	total: 1m 1s	remaining: 15.2s
900:	total: 1m 7s	remaining: 7.42s
999:	total: 1m 14s	remaining: 0us


In [82]:
val['y_score'] = model.predict_proba(val[X])[:,1]

# Подбор threshold с помощью валидационной выборки

In [83]:
result = []
for thr in np.linspace(0,1,101):
    val['y_pred'] = val['y_score'] > thr
    f1 = f1_score(val[y],val['y_pred'])
    result.append((thr,f1))

In [84]:
f1_df = pd.DataFrame(result,columns=['thr','f1'])

In [89]:
f1_df.sort_values('f1',ascending=False)

,thr,f1
30,0.30,0.631360
31,0.31,0.630537
32,0.32,0.628541
28,0.28,0.628337
29,0.29,0.627451
...,...,...
76,0.76,0.000000
75,0.75,0.000000
74,0.74,0.000000
73,0.73,0.000000


In [93]:
thr = f1_df.sort_values('f1',ascending=False).iloc[0, 0]

In [94]:
thr

0.3

# Проверяем на тестовой выборке

In [95]:
model.fit(train[X], train[y])

0:	total: 52.8ms	remaining: 52.8s
100:	total: 8.14s	remaining: 1m 12s
200:	total: 16.4s	remaining: 1m 5s
300:	total: 24.5s	remaining: 56.9s
400:	total: 33.1s	remaining: 49.4s
500:	total: 41.3s	remaining: 41.2s
600:	total: 49.7s	remaining: 33s
700:	total: 57.9s	remaining: 24.7s
800:	total: 1m 6s	remaining: 16.5s
900:	total: 1m 14s	remaining: 8.19s
999:	total: 1m 22s	remaining: 0us


In [96]:
test['y_score'] = model.predict_proba(test[X])[:,1]

In [97]:
roc_auc_score(test[y],test['y_score'])

0.8051010735758437

In [98]:
test['y_pred'] = test['y_score'] > thr

In [99]:
f1_score(test[y],test['y_pred'])

0.6345524829021708